# Caracterização de eletrofáceis por meio de inteligência artificial com abordagem supervisionada
## Boxplots das propriedades de litologias nos diferentes membros

- **Nome do Arquivo**: boxplots_litologia_membro.ipynb
- **Autor**: Rafael Takeguma Goto
- **Versão**: 1.0
- **Data de Criação**: 08/07/2024
- **Data de Modificação**: 08/07/2024

### Descrição: 
Este notebook contém o a etapa do trabalho referente à criação de boxplots das propriedades (curvas) das litologias em diferentes membros nos poços individuais.

### Leitura do conjunto de dados

O conjunto de dados é constituído por arquivos csv gerados a partir da fusão da extração dos arquivos dlis com os arquivos agp de cada poço.